## Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# tensorflow
import tensorflow as tf
print('Tested with TensorFlow 1.2.0')
print ('Your TensorFlow version:', tf.__version__) # tested with v1.2
from tensorflow.contrib.learn.python.learn.estimators import constants
from tensorflow.contrib.learn.python.learn.estimators.dynamic_rnn_estimator import PredictionType

# Estimators
from tensorflow.contrib import learn

# Model builder
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from tensorflow.contrib.learn.python.learn.estimators import rnn_common


# Input function
from tensorflow.python.estimator.inputs import numpy_io

# data and data preprocessing
from tensorflow.contrib.keras.python.keras.datasets import imdb 
from tensorflow.contrib.keras.python.keras.preprocessing import sequence

# Run an experiment
from tensorflow.contrib.learn.python.learn import learn_runner

# numpy
import numpy as np

# Enable TensorFlow logs
tf.logging.set_verbosity(tf.logging.INFO)

Tested with TensorFlow 1.2.0
Your TensorFlow version: 1.2.1


## Helpers

In [3]:
# map word to index
word_to_index = imdb.get_word_index()
# map index to word
index_to_word = {}
num_words = 0
for k in word_to_index: 
    index_to_word[word_to_index[k]] = k
    num_words += 1

# turn a sequence into a sentence
def get_sentence(seq):
    sentence = ''
    for v in seq:
        if v != 0: # 0 means it was just added to the sentence so it could have maxlen words
            sentence += index_to_word[int(v)] + ' '
    return sentence

# turn a sentence into a sequence
def gen_sequence(sentence):
    seq = []
    for word in sentence:
        seq.append(word_to_index[word])
    return np.asarray(seq, dtype=np.float32)

print('there are', num_words, 'words in the files')
#for i in range(1, 100):
#    print(i, index_to_word[i])
#print(index_to_word[49], index_to_word[212], index_to_word[242], index_to_word[136])

there are 88584 words in the files


## Visualizing data

In [4]:
'''
# ------------------- negative
print('-' * 30)
print('Example of a negative review')
print('-' * 30)

x = open('data/train/neg/0_3.txt')
r = x.readline()
print(r)

# ------------------ positive
print()
print('-' * 30)
print('Example of a positive review')
print('-' * 30)

x = open('data/train/pos/0_9.txt')
r = x.readline()
print(r)
'''

"\n# ------------------- negative\nprint('-' * 30)\nprint('Example of a negative review')\nprint('-' * 30)\n\nx = open('data/train/neg/0_3.txt')\nr = x.readline()\nprint(r)\n\n# ------------------ positive\nprint()\nprint('-' * 30)\nprint('Example of a positive review')\nprint('-' * 30)\n\nx = open('data/train/pos/0_9.txt')\nr = x.readline()\nprint(r)\n"

In [5]:
print('Loading data')
NUM_WORDS = 1000
MAX_LENGTH = 250

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)


Loading data


In [6]:
x_train.size

25000

In [7]:
# parameters
LEARNING_RATE = 0.001
BATCH_SIZE = 16

def get_input_fn(x_in, y_in, shuffle=True, epochs=1):
    def length_bin(length, length_step=10, maxlen=MAX_LENGTH):
        # This function sets the sequence-length bin.
        # The returned value is the length of the longest sequence allowed in the bin.
        bin_id = (length // length_step + 1)*length_step
        return tf.cast(tf.minimum(bin_id, maxlen),tf.int64)
    
    def make_batch(key, ds):
        # Eliminate the extra padding
        ds = ds.map(lambda x,x_len,y: (x[:tf.cast(key,tf.int32)],x_len, y))
        
        # convert the entire contents of the bin to a batch
        ds = ds.batch(BATCH_SIZE)
        return ds
        
    
    def input_fn():
        ys = tf.contrib.data.Dataset.from_tensor_slices(y_in)
        
        x_len = np.minimum(np.array([len(seq) for seq in x_in]), MAX_LENGTH).astype('int32')
        
        # DynalicRNNEstimator uses `rnn_common.select_last_activations`
        #     https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/contrib/learn/python/learn/estimators/dynamic_rnn_estimator.py
        # So we want to add padding at the end of the sequence, the default is the start of the sequence:
        #     https://www.tensorflow.org/versions/master/api_docs/python/tf/contrib/keras/preprocessing/sequence/pad_sequences
        xs = sequence.pad_sequences(x_in, maxlen=MAX_LENGTH, padding='post')
        
        ds = tf.contrib.data.Dataset.from_tensor_slices((xs, x_len, y_in))
        
        ds = ds.repeat(epochs)
        
        if shuffle:
            ds = ds.shuffle(10000)

        ds = ds.group_by_window(
            key_func=lambda x, x_len, y: length_bin(x_len), 
            reduce_func=make_batch,
            window_size=BATCH_SIZE)
        
        x,x_len,y = ds.make_one_shot_iterator().get_next()

        return {'x':x, rnn_common.RNNKeys.SEQUENCE_LENGTH_KEY:x_len},y
        
    return input_fn

In [8]:
train_input_fn = get_input_fn(x_train, y_train, BATCH_SIZE, epochs=100)
test_input_fn = get_input_fn(x_test, y_test, BATCH_SIZE, epochs=1)

In [9]:
x,y = train_input_fn()

with tf.train.MonitoredSession() as sess:
    for n in range(30):
        x_,y_ = sess.run([x,y])

In [10]:
x

{'sequence_length': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>,
 'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?) dtype=int32>}

In [11]:
x_

{'sequence_length': array([190, 193, 190, 193, 191, 190, 194, 192, 191, 197, 195, 194, 197,
        190, 198, 192], dtype=int32),
 'x': array([[  1,  13, 124, ...,   0,   0,   0],
        [  1,  54,  13, ...,   0,   0,   0],
        [  1, 348,  94, ...,   0,   0,   0],
        ..., 
        [  1, 260, 332, ...,   0,   0,   0],
        [  1,  51, 630, ...,   2,   0,   0],
        [  1,   2,   2, ...,   0,   0,   0]], dtype=int32)}

In [12]:
x_['x'].shape

(16, 200)

In [13]:
x_['x']

array([[  1,  13, 124, ...,   0,   0,   0],
       [  1,  54,  13, ...,   0,   0,   0],
       [  1, 348,  94, ...,   0,   0,   0],
       ..., 
       [  1, 260, 332, ...,   0,   0,   0],
       [  1,  51, 630, ...,   2,   0,   0],
       [  1,   2,   2, ...,   0,   0,   0]], dtype=int32)

In [14]:
y

<tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=int64>

In [15]:
xc = tf.contrib.layers.sparse_column_with_integerized_feature('x',NUM_WORDS)
xc = tf.contrib.layers.embedding_column(xc,30)

In [16]:

estimator = tf.contrib.learn.DynamicRnnEstimator(
     model_dir='tensorboard/RNN_dropout_2',
     problem_type = constants.ProblemType.CLASSIFICATION,
     prediction_type = PredictionType.SINGLE_VALUE,
     sequence_feature_columns = [xc],
     context_feature_columns = None,
     num_units = [256, 128],
     cell_type = 'lstm', 
     optimizer = 'Adam',
     learning_rate = LEARNING_RATE,
     num_classes = 2,
     dropout_keep_probabilities=[0.9,0.9,0.9])

# create experiment
def generate_experiment_fn():
  
  """
  Create an experiment function given hyperparameters.
  Returns:
    A function (output_dir) -> Experiment where output_dir is a string
    representing the location of summaries, checkpoints, and exports.
    this function is used by learn_runner to create an Experiment which
    executes model code provided in the form of an Estimator and
    input functions.
    All listed arguments in the outer function are used to create an
    Estimator, and input functions (training, evaluation, serving).
    Unlisted args are passed through to Experiment.
  """

  def _experiment_fn(output_dir):

    train_input = train_input_fn
    test_input = test_input_fn
    
    return tf.contrib.learn.Experiment(
        estimator,
        train_input_fn=train_input,
        eval_input_fn=test_input,
    )
  return _experiment_fn

# run experiment 
learn_runner.run(generate_experiment_fn(), output_dir='/tmp/sentiment_analysis')

Instructions for updating:
This file will be removed after the deprecation date.Please switch to third_party/tensorflow/contrib/learn/python/learn/estimators/head.py
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9cf77ade10>, '_task_type': None, '_is_chief': True, '_num_ps_replicas': 0, '_session_config': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_environment': 'local', '_task_id': 0, '_master': '', '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_model_dir': 'tensorboard/RNN_dropout_2', '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 0}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


/usr/local/lib/python3.4/dist-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into tensorboard/RNN_dropout_2/model.ckpt.
INFO:tensorflow:step = 1, loss = 0.69313
INFO:tensorflow:Starting evaluation at 2017-07-05-23:56:36
INFO:tensorflow:Restoring parameters from tensorboard/RNN_dropout_2/model.ckpt-1
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:

INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [54/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [63/100]
INFO:tensorflow:Evaluation [64/100]
INFO:tensorflow:Evaluation [65/100]
INFO:tensorflow:Evaluation [66/100]
INFO:tensorflow:Evaluation [67/100]
INFO:tensorflow:Evaluation [68/100]
INFO:tensorflow:Evaluation [69/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [71/100]
INFO:tensorflow:Evaluation [72/100]
INFO:tensorflow:Evaluation [73/100]
INFO:tensorflow:Evaluation [74/100]
INFO:tensorflow:Evaluation [75/100]
INFO:tensorflow:Evaluation [76/100]
INFO:tensorflow:Evaluation [77/100]
INFO:tensorflow:Evaluation [78/100]
INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [

...

INFO:tensorflow:global_step/sec: 4.36149
INFO:tensorflow:step = 147401, loss = 0.000178408 (22.928 sec)
INFO:tensorflow:global_step/sec: 4.07316
INFO:tensorflow:step = 147501, loss = 3.93274e-05 (24.551 sec)
INFO:tensorflow:global_step/sec: 4.40875
INFO:tensorflow:step = 147601, loss = 0.00235749 (22.682 sec)
INFO:tensorflow:global_step/sec: 4.21892
INFO:tensorflow:step = 147701, loss = 0.000416975 (23.703 sec)
INFO:tensorflow:global_step/sec: 3.98248
INFO:tensorflow:step = 147801, loss = 0.00107831 (25.110 sec)
INFO:tensorflow:global_step/sec: 4.36817
INFO:tensorflow:step = 147901, loss = 0.000157842 (22.893 sec)
INFO:tensorflow:global_step/sec: 4.35921
INFO:tensorflow:step = 148001, loss = 0.00701421 (22.940 sec)
INFO:tensorflow:global_step/sec: 4.09014
INFO:tensorflow:step = 148101, loss = 0.000524412 (24.449 sec)
INFO:tensorflow:global_step/sec: 4.34701
INFO:tensorflow:step = 148201, loss = 0.00815939 (23.004 sec)
INFO:tensorflow:global_step/sec: 4.10233
INFO:tensorflow:step = 1483

INFO:tensorflow:Starting evaluation at 2017-07-06-09:36:43
INFO:tensorflow:Restoring parameters from tensorboard/RNN_dropout_2/model.ckpt-151030
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]


INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [54/100]
INFO:tensorflow:Evaluation [55/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [69/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [71/100]
INFO:tensorflow:Evaluation [72/100]
INFO:tensorflow:Evaluation [73/100]
INFO:tensorflow:Evaluation [74/100]
INFO:tensorflow:Evaluation [75/100]
INFO:tensorflow:Evaluation [76/100]
INFO:tensorflow:Evaluation [77/100]
INFO:tensorflow:Evaluation [78/100]
INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [

({'accuracy': 0.86312503, 'global_step': 156260, 'loss': 0.7407614}, [])

In [ ]:
estimator = tf.contrib.learn.DynamicRnnEstimator(
     model_dir='tensorboard/RNN',
     problem_type = constants.ProblemType.CLASSIFICATION,
     prediction_type = PredictionType.SINGLE_VALUE,
     sequence_feature_columns = [xc],
     context_feature_columns = None,
     num_units = [256, 128],
     cell_type = 'lstm', 
     optimizer = 'Adam',
     learning_rate = LEARNING_RATE,
     num_classes = 2)

In [ ]:
# generate predictions
preds = list(estimator.predict(input_fn=test_input_fn))

# number of outputs we want to see the prediction
NUM_EVAL = 10
def check_prediction(x, y, p, index):
    print('prediction:', p[index]['classes'])
    print('target:', y[index])
    print('sentence:', get_sentence(x[index]))

for i in range(NUM_EVAL):
    index = np.random.randint(limit)
    print('test:', index)
    print('-' * 30)
    print(np.asarray(x_test[index], dtype=np.int32))
    check_prediction(x_test, y_test, preds, index)
    print()